# Python标准库系列之Memcache模块

https://blog.ansheng.me/article/python-standard-library-memcache/

这个模块是一个Python操作memcached的一个API接口。

## What is Memcached?

Free & open source, high-performance, distributed memory object caching system, generic in nature, but intended for use in speeding up dynamic web applications by alleviating database load.

Memcached is an in-memory key-value store for small chunks of arbitrary data (strings, objects) from results of database calls, API calls, or page rendering.

Memcached is simple yet powerful. Its simple design promotes quick deployment, ease of development, and solves many problems facing large data caches. Its API is available for most popular languages.

## Install on macOs
brew install memcached

In [1]:
! cat '/tmp/memcached.pid'

49250


In [2]:
#先查看network端口是否启动
! lsof -nP -iTCP -sTCP:LISTEN | grep memcached

memcached 49250 binyang   21u  IPv4 0x6b5226e14209293f      0t0  TCP *:11211 (LISTEN)


#### Use lsof instead of 'netstat' on macOs
https://tonydeng.github.io/2016/07/07/use-lsof-to-replace-netstat/

## Memcache使用

安装Memcache

In [3]:
import memcache
conn = memcache.Client(['0.0.0.0:11211'])

In [4]:
conn.set('k1', 'v1')

True

In [5]:
conn.get('k1')

'v1'

In [6]:
# 设置超时时间 timeout was set to 5
conn.set('k', 'v', 5)

True

In [7]:
conn.get('k')

'v'

In [8]:
# add new 'k' failed
conn.add('k','hello')

False

In [9]:
conn.get('k')

'v'

In [10]:
# wait until 'k' timeout
import time
time.sleep(6)

In [11]:
# add new 'k' succeeded
conn.add('k','hello')

True

In [12]:
conn.get('k')

'hello'

In [13]:
conn.replace('k','helloworld')

True

In [14]:
conn.get('k')

'helloworld'

In [15]:
# replace failed when the value doesn't exist
conn.replace('kkkk','hello')

False

In [16]:
# set multiple values
conn.set_multi({'key1':'value1','key2':'value2'})

[]

In [17]:
conn.get('key1')

'value1'

In [18]:
# delete
conn.delete('key1')

1

In [19]:
conn.get('key1')

In [20]:
# delete multiple values
conn.delete_multi(['key3', 'key4'])

1

In [21]:
# get multiple values
conn.get_multi(['key5','key6'])

{}

自增与自减，将Memcached中的某一个值加或减少N(N默认为1)

In [22]:
conn.set('number','9')

True

In [23]:
conn.incr('number')

10

In [24]:
conn.incr('number', 10)

20

In [25]:
conn.decr('number')

19

In [26]:
conn.decr('number', 10)

9

### locking and block

现在A用户和B用户同时获取到了这两个值，如果有其中的任何一个用户对这个值进行了修改，那么另外一个用户在对这个值进行操作的时候就会报错。

如果要解决以上的问题可以使用gets与cas

In [27]:
# -- s1.py
# _*_ coding:utf-8 _*_
import memcache
import time
conn1 = memcache.Client(['0.0.0.0:11211'], debug=True, cache_cas=True)
conn1.set('n', 9)
result1 = conn1.gets('n')
conn2 = memcache.Client(['0.0.0.0:11211'], debug=True, cache_cas=True)
result2 = conn2.gets('n')
print('result1: ',result1)
print('result2: ',result2)
time.sleep(1)
conn1.cas('n', 99)
print('after 1 sec, conn1.gets : ',conn1.gets('n'))
time.sleep(1)
conn2.cas('n', 100)
print('after 1 sec, conn2.gets : ',conn2.gets('n'))

result1:  9
result2:  9
after 1 sec, conn1.gets :  99
after 1 sec, conn2.gets :  99


MemCached: while expecting 'STORED', got unexpected response 'EXISTS'
